In [88]:
import pandas as pd
from sklearn import preprocessing
import numpy as np
import statsmodels.formula.api as sm
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_columns', 500)


In [254]:
df = pd.read_csv('../data/clean_train_all_cols_chase.csv', parse_dates=['timestamp'], index_col="id")  
# test = pd.read_csv('../data/clean_test_all_cols_chase.csv', parse_dates=['timestamp'], index_col="id")

In [255]:
macro = pd.read_csv('../data/macro_chase.csv')
macro['quarter'] = pd.PeriodIndex(macro['Unnamed: 0'], freq='Q').strftime('Q%q-%y')
df['quarter'] = pd.PeriodIndex(df['timestamp'], freq='Q').strftime('Q%q-%y')

df = pd.merge(df,macro[['quarter','nominal_index']], how="left", on="quarter").reset_index(drop=True).set_index(df.index)

cols = ['material', 'build_year', 'num_room', 'state']
print df.shape
df.drop(cols,axis=1,inplace=True)
print df.shape

(29096, 299)
(29096, 295)


In [256]:
# drop all columns that don't have atleast 28000 values
df.dropna(thresh=28000, axis=1, inplace=True)
print df.shape

(29096, 260)


In [257]:
df.dropna(axis=0,inplace=True)
print df.shape

(28090, 260)


In [258]:
# cleaning up the names of the neighborhoods
df.sub_area = df.sub_area.apply(lambda x: x.replace(" ", "_"))
df.sub_area = df.sub_area.apply(lambda x: x.replace("'", ""))
df.sub_area = df.sub_area.apply(lambda x: x.replace("-", ""))

In [259]:
df['kitch_to_life'] = df.kitch_sq / df.life_sq
df['life_to_full'] = df.life_sq / df.full_sq
df['walk_up_penalty'] = 0
df.loc[(df.floor>4) & (df.max_floor < 6),'walk_up_penalty'] = 1 

In [260]:
df['price_doc'] = df.price_doc / df.nominal_index
df['price_full'] = df.price_doc / df.full_sq
# df['log_price'] = np.log(df.price_doc)

# creating a variable for how expensive the property is
df['price_class'] = pd.qcut(df.price_full, 5, labels=["E","D","C","B","A"])

## PCA

In [261]:
from sklearn.model_selection import train_test_split

In [290]:
from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA


drop_cols = ['timestamp','price_doc','price_full','okurg_district','sub_area','quarter']
keep_cols = ['price_class','kremlin_km','floor','full_sq','product_type','full_sq','kitch_sq','max_floor','life_sq']


df_obj = df.drop(drop_cols,axis=1).select_dtypes(include=['object'])
df_num = df.drop(drop_cols,axis=1).select_dtypes(exclude=['object'])

# df = df[keep_cols]
# df_obj = df.select_dtypes(include=['object'])
# df_num = df.select_dtypes(exclude=['object'])

dummies = pd.get_dummies(df_obj)
df_all = pd.concat([df_num,dummies],axis=1)